In [1]:
from keplergl import KeplerGl
import numpy as np
import pandas as pd
from visvalingamwyatt import Simplifier
import json

~*~ Must open in jupyter notebook for widget to work ~*~

# Neighborhoods

In [2]:
bounds = pd.read_pickle('community_boundaries.pickle')

In [3]:
daily_df = pd.read_pickle('daily_df.pickle')

daily_df.head(1)

,date,pickup_community_area,rides
32801,2019-12-31,9,414


In [4]:
hour_test = pd.read_pickle('hour_test.pickle')

hour_test.head(1)

,date_time,pickup_community_area,rides
29449,2018-11-17 01:00:00,1.0,117


In [5]:
def simplify_coords(coords, ratio=.5):
    
    simplifier = Simplifier(coords['coordinates'][0][0])
    coords = {'type': 'MultiPolygon',
              'coordinates': [[[list(x) for x in list(simplifier.simplify(ratio=ratio))]]]}
    
    return coords

In [6]:
bounds['simple'] = bounds['the_geom'].apply(lambda row: simplify_coords(row, ratio=.05))

In [7]:
orig = np.array([len(x['coordinates'][0][0]) for x in bounds['the_geom'][:10]])
one = np.array([len(x['coordinates'][0][0]) for x in bounds['simple'][:10]])

print(f'length differences: {orig- one}')
print(f'original lengths:   {orig}')
print(f'new lengths:        {one}')

length differences: [334 298 300 311 658 740 289 693 874 879]
original lengths:   [352 314 317 328 693 779 305 730 921 926]
new lengths:        [18 16 17 17 35 39 16 37 47 47]


In [8]:
to_keep = ['area_numbe','community', 'simple']

merged = hour_test.merge(bounds[to_keep], how='left',
                        left_on='pickup_community_area',
                        right_on='area_numbe')

merged['date_time'] = merged['date_time'].map(lambda x: x.isoformat())

In [9]:
def open_config(file_name='config.json'):
    
    try:
        current_file = json.load(open(file_name, 'r'))
        print('config loaded')
        return current_file
    except FileNotFoundError:
        print('no current config file. plz initialize map, and use map.config')
        pass

In [10]:
config = open_config()

config loaded


In [11]:
map_1 = KeplerGl(height=800)
map_1.add_data(merged, 'data_1')
map_1

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'data_1': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, …

In [17]:
def save_config(config, file_name='config.json'):
    
    try:
        current_file = json.load(open(file_name, 'r'))
        if config != current_file:
            print('changes detected. saving current config')
            with open(file_name, 'w') as f:
                json.dump(config, f)
        else:
            print('no changes detected')
            
    except FileNotFoundError:
        print('no existing file in directory. saving current config')
        with open(file_name, 'w') as f:
                json.dump(config, f)

In [18]:
save_config(map_1.config)

changes detected. saving current config


In [19]:
map_1.save_to_html(file_name='chicago.html')

Map saved to chicago.html!
